# scratchpad

for sketching out some ideas

In [1]:
from ophyd import Device, Component, EpicsSignal

In [2]:
class UserDataDevice(Device):
    FS_order_number = Component(EpicsSignal,    "USAXS:FS_OrderNumber")
    GUP_number = Component(EpicsSignal,         "GUPNumber")
    macro_file_time = Component(EpicsSignal,    "USAXS:macroFileTime")
    run_cycle = Component(EpicsSignal,          "RunCycle")
    sample_thickness = Component(EpicsSignal,   "USAXS:SampleThickness")
    sample_title = Component(EpicsSignal,       "USAXS:sampleTitle")
    spec_file = Component(EpicsSignal,          "USAXS:specFile")
    spec_scan = Component(EpicsSignal,          "USAXS:specScan")
    state = Component(EpicsSignal,              "USAXS:state")
    time_stamp = Component(EpicsSignal,         "USAXS:timeStamp")
    user_dir = Component(EpicsSignal,           "USAXS:userDir")
    user_name = Component(EpicsSignal,          "UserName")
    
    # FIXME: comment needs fixing
    # for GUI to know if user is collecting data: 1-not running, 0 running ???
    # for GUI to know if user is collecting data: 1=running, 0=not running
    collection_in_progress = Component(EpicsSignal, "dataColInProgress")


user_data = UserDataDevice("9idcLAX:", name="user_data")

In [4]:
user_data.read()

OrderedDict([('user_data_FS_order_number',
              {'value': 1, 'timestamp': 1533241697.351012}),
             ('user_data_GUP_number',
              {'value': '57504', 'timestamp': 631152000.0}),
             ('user_data_macro_file_time',
              {'value': 'Thu Aug 02 15:39:25 2018',
               'timestamp': 1533242365.895038}),
             ('user_data_run_cycle',
              {'value': '2018-02', 'timestamp': 631152000.0}),
             ('user_data_sample_thickness',
              {'value': 0.0, 'timestamp': 631152000.0}),
             ('user_data_sample_title',
              {'value': 'Q0003Optics_FlatField',
               'timestamp': 1533242241.676367}),
             ('user_data_spec_file',
              {'value': '07_31_ImgSetup.dat', 'timestamp': 1533242241.6097}),
             ('user_data_spec_scan',
              {'value': 1048, 'timestamp': 1533242241.6097}),
             ('user_data_state',
              {'value': 'USAXS macro file done',
               'ti

In [8]:
user_data.collection_in_progress.enum_strs

('On', 'Off')

In [17]:
from ophyd import EpicsMotor

class EpicsMotorLimitsMixin(Device):
    """
    add motor record HLM & LLM fields & compatibility get_lim() and set_lim()
    """
    
    soft_limit_lo = Component(EpicsSignal, ".LLM")
    soft_limit_hi = Component(EpicsSignal, ".HLM")
    
    def get_lim(self, flag):
        """
        Returns the user limit of motor
        
        flag > 0: returns high limit
        flag < 0: returns low limit
        flag == 0: returns None
        """
        if flag > 0:
            return self.high_limit
        else:
            return self.low_limit
    
    def set_lim(self, low, high):
        """
        Sets the low and high limits of motor
        
        * Low limit is set to lesser of (low, high)
        * High limit is set to greater of (low, high)
        * No action taken if motor is moving. 
        """
        if not self.moving:
            self.soft_limit_lo.put(min(low, high))
            self.soft_limit_hi.put(max(low, high))


class UsaxsMotor(EpicsMotor, EpicsMotorLimitsMixin): pass

waxsx = UsaxsMotor('9idcLAX:m58:c0:m4', name='waxsx', labels=("waxs",))  # WAXS X

In [18]:
waxsx.get_lim(1)

255.4

In [19]:
waxsx.get_lim(-1)

-21.00000000000003

In [20]:
waxsx.soft_limit_lo.value

-21.00000000000003

In [21]:
UsaxsSaxsMode = EpicsSignal("9idcLAX:USAXS_Pin:USAXSSAXSMode", name="UsaxsSaxsMode")

In [22]:
UsaxsSaxsMode.value

5.0

In [23]:
UsaxsSaxsMode.value == 5

True